In [ ]:
# Import data into a pandas dataframe 
import pandas as pd 
yeast_dataset = pd.read_csv("yeast.data", header=None)
yeast_dataset.info()

In [ ]:
yeast_dataset.iloc[:,-1] = yeast_dataset.iloc[:,-1].apply(lambda x : {
    "CYT" : 0,
    "NUC" : 1,
    "MIT" : 2,
    "ME3" : 3,
    "ME2" : 4,
    "ME1" : 5, 
    "EXC" : 6,
    "VAC" : 7,
    "POX" : 8,
    "ERL" : 9
}[x])

In [ ]:
yeast_dataset = yeast_dataset.iloc[:,1:]
yeast_dataset.head()

In [ ]:
# Split data to train (80%) and test (20%) data 
from sklearn.model_selection import train_test_split
X = yeast_dataset.iloc[:,:-1]
y = yeast_dataset.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [ ]:
# Launch tpot training 
from tpot import TPOTClassifier
clf_tpot = TPOTClassifier(verbosity=2, max_time_mins=30 )
clf_tpot.fit(X_train, y_train)

In [ ]:
# Launch autosklearn 
import autosklearn.classification
clf_autosklearn = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=1800)
clf_autosklearn.fit(X_train, y_train)

In [ ]:
from oboe import AutoLearner, error
import numpy as np 
x_train = np.array(X_train)
yy_train = np.array(y_train)
method = 'Oboe' # 'Oboe' or 'TensorOboe'
problem_type = 'classification'
clf_oboe = AutoLearner(p_type=problem_type, runtime_limit=30, method=method, verbose=False)
clf_oboe.fit(x_train, yy_train)

In [ ]:
from sklearn.metrics import accuracy_score
print("tpot score:\t\t ", clf_tpot.score(X_test, y_test))
print("autosklern score:\t\t ", clf_autosklearn.score(X_test, y_test))

x_test = np.array(X_test)
yy_test = np.array(y_test)
y_predicted = clf_oboe.predict(x_test)
y_predicted = y_predicted.reshape((yy_test.size,))
print("oboe score:\t\t ", accuracy_score(y_test, y_predicted) )

In [ ]:
import pickle
pickle.dump(clf_autosklearn, open('autosklearn_yeast.sav', 'wb'))
pickle.dump(clf_oboe, open('oboe_yeast.sav', 'wb'))
clf_tpot.export('tpot_yeast.sav')